In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from obspy import read
from obspy.core.utcdatetime import UTCDateTime
import os

#  KELAS DETEKTOR STA/LTA
class STALTADetector:
    """
    Kelas ini berfungsi untuk melakukan proses inti STA/LTA:
    - Memuat file MiniSEED
    - Menghitung STA, LTA, dan rasio STA/LTA
    - Menentukan waktu datang (arrival) gelombang
    - Menyimpan hasil deteksi
    """
    def __init__(self):
        self.trace = None
        self.times = None
        self.sta = None
        self.lta = None
        self.ratio = None
        self.triggers = []
        self.picks = []
        self.last_thresh_on = 3.0
        self.last_thresh_off = 1.0

    def load_mseed(self, filepath):
        """Memuat file MiniSEED dan melakukan pra-pemrosesan."""
        try:
            st = read(filepath)
            if len(st) == 0:
                raise ValueError("File MiniSEED kosong")

            self.trace = st[0]
            self.trace.detrend("linear")
            self.trace.filter("bandpass", freqmin=1.0, freqmax=10.0)
            self.times = self.trace.times("utcdatetime")
            return True
        except Exception as e:
            messagebox.showerror("Error", f"Gagal load file: {str(e)}")
            return False

    def compute_sta_lta(self, sta_sec=1.0, lta_sec=30.0, thresh_on=3.0, thresh_off=1.0):
        """Menghitung nilai STA, LTA, dan rasio STA/LTA untuk mendeteksi gelombang."""
        if self.trace is None:
            return False
        
        data = np.abs(self.trace.data)
        fs = self.trace.stats.sampling_rate
        nsta = int(sta_sec * fs)
        nlta = int(lta_sec * fs)

        if nsta >= len(data) or nlta >= len(data):
            messagebox.showwarning("Parameter", "Window STA/LTA terlalu besar untuk data")
            return False

        # Hitung cumulative sum agar perhitungan moving average cepat
        cumsum = np.cumsum(data)
        sta = np.zeros(len(data))
        lta = np.zeros(len(data))
        
        sta[nsta:] = cumsum[nsta:] - cumsum[:-nsta]
        sta[:nsta] = cumsum[:nsta]
        sta /= nsta

        lta[nlta:] = cumsum[nlta:] - cumsum[:-nlta]
        lta[:nlta] = cumsum[:nlta]
        lta /= nlta
        
        lta[lta == 0] = 1e-10
        
        self.sta = sta
        self.lta = lta
        self.ratio = sta / lta

        # Simpan nilai threshold terakhir
        self.last_thresh_on = thresh_on
        self.last_thresh_off = thresh_off

        # Deteksi waktu trigger
        self.triggers = []
        self.picks = []
        in_trigger = False

        for i in range(len(self.ratio)):
            if not in_trigger and self.ratio[i] > thresh_on:
                on_time = self.times[i]
                self.picks.append(on_time)
                in_trigger = True
            elif in_trigger and self.ratio[i] < thresh_off:
                off_time = self.times[i]
                self.triggers.append((on_time, off_time))
                in_trigger = False
        
        return True

    def plot(self, ax_trace, ax_stalta, ax_ratio):
        """Menampilkan tiga grafik: sinyal asli, STA-LTA, dan rasio STA/LTA."""
        ax_trace.clear()
        ax_stalta.clear()
        ax_ratio.clear()
        
        if self.trace is None:
            return
        
        # Plot sinyal asli
        ax_trace.plot(self.times, self.trace.data, color='black', linewidth=0.8)
        ax_trace.set_ylabel("Amplitudo")
        ax_trace.set_title("Sinyal Asli")
        ax_trace.grid(True)
        
        # Plot STA dan LTA
        ax_stalta.plot(self.times, self.sta, label='STA', color='blue')
        ax_stalta.plot(self.times, self.lta, label='LTA', color='red')
        ax_stalta.set_ylabel("Nilai")
        ax_stalta.set_title("STA (biru) & LTA (merah)")
        ax_stalta.legend()
        ax_stalta.grid(True)
        
        # Plot rasio STA/LTA
        ax_ratio.plot(self.times, self.ratio, label='STA/LTA', color='green')

        # Garis ambang threshold ON dan OFF 
        ax_ratio.axhline(self.last_thresh_on, color='red', linestyle='--', linewidth=1.2,
                         label=f'Threshold ON ({self.last_thresh_on})')
        ax_ratio.axhline(self.last_thresh_off, color='orange', linestyle='--', linewidth=1.2,
                         label=f'Threshold OFF ({self.last_thresh_off})')

        # Area waktu trigger
        for on_time, off_time in self.triggers:
            ax_ratio.axvspan(on_time, off_time, color='yellow', alpha=0.3)
        for pick in self.picks:
            ax_ratio.axvline(pick, color='red', linestyle='--', linewidth=1.2)
            ax_trace.axvline(pick, color='red', linestyle='--', linewidth=1.2)
            ax_stalta.axvline(pick, color='red', linestyle='--', linewidth=1.2)
        
        ax_ratio.set_ylabel("Ratio")
        ax_ratio.set_xlabel("Waktu (UTC)")
        ax_ratio.set_title(f"STA/LTA Ratio | Picks: {len(self.picks)}")
        ax_ratio.legend()
        ax_ratio.grid(True)


#  KELAS GUI APLIKASI
class STALTAApp(tk.Tk):
    """Aplikasi GUI STA/LTA untuk deteksi waktu datang gelombang."""
    def __init__(self):
        super().__init__()
        self.title("STA/LTA Detector - Deteksi Waktu Datang Gelombang")
        self.geometry("1200x800")
        self.detector = STALTADetector()
        self.create_widgets()
        
    def create_widgets(self):
        """Membuat seluruh elemen GUI."""
        control_frame = ttk.Frame(self, padding=10)
        control_frame.pack(side=tk.LEFT, fill=tk.Y)
        
        ttk.Button(control_frame, text="Load File MiniSEED", command=self.load_file).pack(fill=tk.X, pady=5)
        
        ttk.Label(control_frame, text="STA (detik):").pack(pady=2)
        self.entry_sta = ttk.Entry(control_frame)
        self.entry_sta.insert(0, "1.0")
        self.entry_sta.pack(fill=tk.X)
        
        ttk.Label(control_frame, text="LTA (detik):").pack(pady=2)
        self.entry_lta = ttk.Entry(control_frame)
        self.entry_lta.insert(0, "30.0")
        self.entry_lta.pack(fill=tk.X)
        
        ttk.Label(control_frame, text="Threshold ON:").pack(pady=2)
        self.entry_on = ttk.Entry(control_frame)
        self.entry_on.insert(0, "3.0")
        self.entry_on.pack(fill=tk.X)
        
        ttk.Label(control_frame, text="Threshold OFF:").pack(pady=2)
        self.entry_off = ttk.Entry(control_frame)
        self.entry_off.insert(0, "1.0")
        self.entry_off.pack(fill=tk.X)
        
        ttk.Button(control_frame, text="Compute STA/LTA", command=self.compute).pack(fill=tk.X, pady=10)
        ttk.Button(control_frame, text="Save Plot (PNG)", command=self.save_plot).pack(fill=tk.X, pady=5)
        ttk.Button(control_frame, text="Save Picks (TXT)", command=self.save_picks).pack(fill=tk.X, pady=5)
        
        self.label_info = ttk.Label(control_frame, text="Belum ada data")
        self.label_info.pack(pady=10)
        
        plot_frame = ttk.Frame(self)
        plot_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        self.fig, (self.ax1, self.ax2, self.ax3) = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
        self.fig.tight_layout(pad=3.0)
        self.canvas = FigureCanvasTkAgg(self.fig, plot_frame)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    def load_file(self):
        """Memuat file MiniSEED dan menampilkan info dasarnya."""
        filepath = filedialog.askopenfilename(
            title="Pilih file MiniSEED",
            filetypes=[("MiniSEED files", "*.mseed *.MSEED"), ("All files", "*.*")]
        )
        if filepath:
            if self.detector.load_mseed(filepath):
                self.label_info.config(text=f"Loaded: {os.path.basename(filepath)}\n"
                                          f"Channel: {self.detector.trace.stats.channel}\n"
                                          f"Start: {self.detector.trace.stats.starttime}\n"
                                          f"Samp. Rate: {self.detector.trace.stats.sampling_rate} Hz")
                self.compute()
    
    def compute(self):
        """Mengambil input dari GUI dan menghitung STA/LTA."""
        if self.detector.trace is None:
            messagebox.showwarning("Data", "Load file dulu!")
            return
        
        try:
            sta = float(self.entry_sta.get())
            lta = float(self.entry_lta.get())
            thr_on = float(self.entry_on.get())
            thr_off = float(self.entry_off.get())
        except ValueError:
            messagebox.showerror("Input", "Parameter harus angka!")
            return
        
        if self.detector.compute_sta_lta(sta, lta, thr_on, thr_off):
            self.detector.plot(self.ax1, self.ax2, self.ax3)
            self.canvas.draw()
            self.label_info.config(text=f"{self.label_info.cget('text').split('Picks')[0]}\n"
                                      f"Picks terdeteksi: {len(self.detector.picks)}")
    
    def save_plot(self):
        """Menyimpan hasil plot sebagai file PNG."""
        if self.detector.trace is None:
            return
        filepath = filedialog.asksaveasfilename(defaultextension=".png",
                                                filetypes=[("PNG", "*.png")])
        if filepath:
            self.fig.savefig(filepath, dpi=200)
            messagebox.showinfo("Saved", f"Plot disimpan: {filepath}")
    
    def save_picks(self):
        """Menyimpan waktu pick (arrival) ke file TXT."""
        if not self.detector.picks:
            messagebox.showwarning("Picks", "Belum ada pick!")
            return
        filepath = filedialog.asksaveasfilename(defaultextension=".txt",
                                                filetypes=[("Text", "*.txt")])
        if filepath:
            with open(filepath, "w") as f:
                f.write("STA/LTA Picks (Waktu Datang Gelombang)\n")
                f.write(f"Mulai Data: {self.detector.trace.stats.starttime}\n")
                f.write(f"STA: {self.entry_sta.get()}s | LTA: {self.entry_lta.get()}s\n")
                f.write(f"Threshold ON/OFF: {self.entry_on.get()}/{self.entry_off.get()}\n\n")
                f.write("No.\tWaktu (UTC)\tTimestamp\n")
                for i, pick in enumerate(self.detector.picks, 1):
                    f.write(f"{i}\t{pick}\t{pick.timestamp}\n")
            messagebox.showinfo("Saved", f"Picks disimpan: {filepath}")


#  EKSEKUSI PROGRAM
if __name__ == "__main__":
    app = STALTAApp()
    app.mainloop()


KeyboardInterrupt: 